# 101 Calculate AMOC #

In [1]:
%load_ext autoreload
%autoreload 2
    
import os
import sys
import pandas as pd
import xarray as xr
import numpy as np
from dask.distributed import Client
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from pathlib import Path
sys.path.append('/home/563/sc1326/repos/cdrmip_extremes')
from cdrmip_extremes.configs import data_dir, models, expts
from cdrmip_extremes import load_esgf

In [2]:
client = Client(memory_limit=None,threads_per_worker=1,n_workers=28)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/33149/status,
Dashboard: /proxy/33149/status,Workers: 28
Total threads: 28,Total memory: 0 B
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37917,Workers: 0
Dashboard: /proxy/33149/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:41133,Total threads: 1
Dashboard: /proxy/33511/status,Memory: 0 B
Nanny: tcp://127.0.0.1:46857,


## Load data ##

### Load 1pctCO2-cdr data ###

In [3]:
msftmz_search_down = load_esgf.esgf_search(
    activity_id='CDRMIP',
    experiment_id='1pctCO2-cdr',
    table_id='Omon',
    variable_id='msftmz'
)

msftyz_search_down = load_esgf.esgf_search(
    activity_id='CDRMIP',
    experiment_id='1pctCO2-cdr',
    table_id='Omon',
    variable_id='msftyz'
)

https://esgf-node.llnl.gov/esg-search/search/?activity_id=CDRMIP&experiment_id=1pctCO2-cdr&table_id=Omon&variable_id=msftmz&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=0
https://esgf-node.llnl.gov/esg-search/search/?activity_id=CDRMIP&experiment_id=1pctCO2-cdr&table_id=Omon&variable_id=msftmz&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=10
https://esgf-node.llnl.gov/esg-search/search/?activity_id=CDRMIP&experiment_id=1pctCO2-cdr&table_id=Omon&variable_id=msftmz&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=20
https://esgf-node.llnl.gov/esg-search/search/?activity_id=CDRMIP&experiment_id=1pctCO2-cdr&table_id=Omon&variable_id=msftmz&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=30
https://esgf-node.llnl.gov/esg-search/search/?activity_id=CDRMIP&experiment_id=1pctCO2-cdr&table_id=Omon&variable_id=msftmz&project=CMIP6&type=File&distrib=false&format=application%

In [4]:
msftmz_search_down

,experiment_id,source_id,member_id,table_id,variable_id,activity_id,grid_label,version,urls
0,1pctCO2-cdr,MIROC-ES2L,r1i1p1f2,Omon,msftmz,CDRMIP,gr,v20210317,[http://esgf-data02.diasjp.net/thredds/dodsC/e...
1,1pctCO2-cdr,ACCESS-ESM1-5,r1i1p1f1,Omon,msftmz,CDRMIP,gn,v20191118,[https://esgf-node.ornl.gov/thredds/dodsC/css0...
2,1pctCO2-cdr,CESM2,r1i1p1f1,Omon,msftmz,CDRMIP,gn,v20190919,[http://esgf-data.ucar.edu/thredds/dodsC/esg_d...
3,1pctCO2-cdr,NorESM2-LM,r1i1p1f1,Omon,msftmz,CDRMIP,grz,v20210118,[https://esgf-node.ornl.gov/thredds/dodsC/css0...
4,1pctCO2-cdr,CanESM5,r1i1p2f1,Omon,msftmz,CDRMIP,gn,v20190429,[http://esgf-node.ornl.gov/thredds/dodsC/css03...


In [5]:
msftyz_models = ['CNRM-ESM2-1','GFDL-ESM4','UKESM1-0-LL']
msftyz_search_down = msftyz_search_down.query('source_id in @msftyz_models')
msftyz_search_down

,experiment_id,source_id,member_id,table_id,variable_id,activity_id,grid_label,version,urls
0,1pctCO2-cdr,CNRM-ESM2-1,r1i1p1f2,Omon,msftyz,CDRMIP,gn,v20210121,[https://esgf-node.ornl.gov/thredds/dodsC/css0...
2,1pctCO2-cdr,GFDL-ESM4,r1i1p1f1,Omon,msftyz,CDRMIP,gn,v20180701,[http://esgdata.gfdl.noaa.gov/thredds/dodsC/gf...
3,1pctCO2-cdr,UKESM1-0-LL,r1i1p1f2,Omon,msftyz,CDRMIP,gn,v20200425,[https://esgf-node.ornl.gov/thredds/dodsC/css0...


In [6]:
msftmz_down = load_esgf.load_mmm_from_search(msftmz_search_down,chunks={'time':12},vars_to_drop='time_bnds')

0it [00:00, ?it/s]

MIROC-ES2L


1it [00:26, 26.97s/it]

ACCESS-ESM1-5


2it [03:29, 118.31s/it]

...

3it [04:00, 78.36s/it] 

NorESM2-LM


4it [05:17, 77.86s/it]

CanESM5



it [05:35, 67.19s/it]

In [7]:
msftyz_down = load_esgf.load_mmm_from_search(msftyz_search_down,chunks={'time':12},vars_to_drop='time_bnds')

0it [00:00, ?it/s]

CNRM-ESM2-1


1it [00:01,  1.88s/it]

GFDL-ESM4


2it [01:22, 48.25s/it]

UKESM1-0-LL



it [01:53, 37.75s/it]

### Load 1pctCO2 data ###

In [14]:
msftmz_search_up = load_esgf.search_mmm_for_parent(msftmz_down)
msftyz_search_up = load_esgf.search_mmm_for_parent(msftyz_down)

https://esgf-node.llnl.gov/esg-search/search/?activity_id=CMIP&experiment_id=1pctCO2&source_id=MIROC-ES2L&member_id=r1i1p1f2&table_id=Omon&variable_id=msftmz&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=0
https://esgf-node.llnl.gov/esg-search/search/?activity_id=CMIP&experiment_id=1pctCO2&source_id=ACCESS-ESM1-5&member_id=r1i1p1f1&table_id=Omon&variable_id=msftmz&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=0
https://esgf-node.llnl.gov/esg-search/search/?activity_id=CMIP&experiment_id=1pctCO2&source_id=ACCESS-ESM1-5&member_id=r1i1p1f1&table_id=Omon&variable_id=msftmz&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=10
https://esgf-node.llnl.gov/esg-search/search/?activity_id=CMIP&experiment_id=1pctCO2&source_id=ACCESS-ESM1-5&member_id=r1i1p1f1&table_id=Omon&variable_id=msftmz&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=20
https://esgf-node.llnl.gov/esg-search

In [15]:
msftmz_search_up

,experiment_id,source_id,member_id,table_id,variable_id,activity_id,grid_label,version,urls
0,1pctCO2,MIROC-ES2L,r1i1p1f2,Omon,msftmz,CMIP,gr,v20210317,[https://esgf-node.ornl.gov/thredds/dodsC/css0...
0,1pctCO2,ACCESS-ESM1-5,r1i1p1f1,Omon,msftmz,CMIP,gn,v20191115,[http://esgf-node.ornl.gov/thredds/dodsC/css03...
0,1pctCO2,CESM2,r1i1p1f1,Omon,msftmz,CMIP,gn,v20190826,[http://esgf-data.ucar.edu/thredds/dodsC/esg_d...
0,1pctCO2,CanESM5,r1i1p2f1,Omon,msftmz,CMIP,gn,v20190429,[http://esgf-node.ornl.gov/thredds/dodsC/css03...


In [16]:
msftyz_search_up

,experiment_id,source_id,member_id,table_id,variable_id,activity_id,grid_label,version,urls
0,1pctCO2,CNRM-ESM2-1,r1i1p1f2,Omon,msftyz,CMIP,gn,v20181018,[http://esgf-node.ornl.gov/thredds/dodsC/css03...
0,1pctCO2,GFDL-ESM4,r1i1p1f1,Omon,msftyz,CMIP,gn,v20180701,[http://esgf-node.ornl.gov/thredds/dodsC/css03...
0,1pctCO2,UKESM1-0-LL,r1i1p1f2,Omon,msftyz,CMIP,gn,v20190821,[http://esgf-node.ornl.gov/thredds/dodsC/css03...


In [17]:
msftmz_up = load_esgf.load_mmm_from_search(msftmz_search_up,chunks={'time':12},vars_to_drop='time_bnds')

0it [00:00, ?it/s]

MIROC-ES2L



it [00:02, ?it/s]

ValueError: Could not find any dimension coordinates to use to order the datasets for concatenation

In [12]:
msftyz_up = load_esgf.load_mmm_from_search(msftyz_search_up,chunks={'time':12},vars_to_drop='time_bnds')

0it [00:00, ?it/s]

CNRM-ESM2-1


1it [00:03,  3.02s/it]

GFDL-ESM4


2it [01:12, 41.94s/it]

UKESM1-0-LL



it [01:23, 27.71s/it]

### Load NorESM2-LM ###

In [ ]:
path = "/scratch/if69/sc1326/CDRMIP/msftmz/NorESM2-LM/1pctCO2/*.nc"
msftmz_up['NorESM2-LM'] = xr.open_mfdataset(
    path,
    chunks={'time':12},
    drop_variables=['time_bnds']
)

## Concatenate ##

In [ ]:
concat = {}
for model in [model for model in models if model not in msftyz_models]:
    ds_up = msftmz_up[model]
    ds_down = msftmz_down[model]
    concat[model] = concat_branches(ds_up,ds_down)

for model in msftyz_models:
    ds_up = msftyz_up[model]
    ds_down = msftyz_down[model]
    concat[model] = concat_branches(ds_up,ds_down)

## Save ##

In [ ]:
save_dir = os.path.join(data_dir,'raw/msftmz')
for model, ds in concat.items():
    if model in msftyz_models:
        var = 'msftyz'
    else:
        var = 'msftmz'
    path = os.path.join(save_dir,f"{model}_cdr-reversibility_{var}.nc")
    ds.to_netcdf(path)